In [3]:
# Import necessary libraries
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(f"API Key: {api_key[:4]}...{api_key[-4:]}")  # Print only the first and last 4 characters for security

API Key: sk-p...O4YA


In [5]:
# Create Instance of OpenAI client
openai = OpenAI()

In [6]:
message = "Hello, how are you?"

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": message}],
)
print(response.choices[0].message.content)

Hello! I am just a computer program, so I don't have feelings in the way humans do. How can I assist you today?


In [7]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class ScrapWebSite:
    def __init__(self, url):
        """
        Create this website object from given url using the BeautifulSoup lib
        """
        self.url = url
        resp = requests.get(url, headers)
        soup = BeautifulSoup(resp.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
            irrelevant.decompose()
        self.text = soup.get_text(separator='\n', strip=True)

In [12]:
scrapedData = ScrapWebSite("https://edwarddonner.com")
print(f"Title: {scrapedData.title}")
print(f"Text snippet: {scrapedData.text[:500]}...")  # Print only the first 500 characters

Title: Home - Edward Donner
Text snippet: Home - Edward Donner
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pur...


In [10]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [16]:
# A function that writes a User Prompt that asks for summaries of scrapedData:

def get_user_prompt(scrapedData):
    user_prompt = f"You are looking at a website titled {scrapedData.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += scrapedData.text
    return user_prompt

In [17]:
print(get_user_prompt(scrapedData))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home - Edward Donner
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
ou

In [21]:
def get_summary(system_prompt, user_prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content
    

In [22]:
print(get_summary(system_prompt, get_user_prompt(scrapedData)))

# Summary of Edward Donner's Website

Edward Donner, a software engineer and tech enthusiast, shares insights into his interests and professional background on his website. He co-founded Nebula.io, a company focused on leveraging AI to help individuals discover their potential and manage talent effectively. With a history of founding the AI startup untapt, which was acquired in 2021, he leads efforts in developing proprietary LLMs tailored for talent acquisition and management.

## News and Announcements
- **September 15, 2025**: "AI in Production: Gen AI and Agentic AI on AWS at scale"
- **May 28, 2025**: "Connecting my courses – become an LLM expert and leader"
- **May 18, 2025**: "2025 AI Executive Briefing"
- **April 21, 2025**: "The Complete Agentic AI Engineering Course"

Edward also expresses a keen interest in DJing and electronic music production, although he humorously mentions being out of practice.
